In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm import tqdm, trange


In [3]:
import torch
if torch.cuda.is_available():  
    device = torch.device("cuda")
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

We will use the GPU: Tesla T4


In [4]:
# Utilities
from time import time
from PIL import Image
from zipfile import ZipFile
import os, sys, itertools, re
import warnings, pickle, string


In [5]:
# Block which runs on both Google Colab and Local PC without any modification
if 'google.colab' in sys.modules:    
    project_path = "/content/drive/My Drive/NER/"
    # Google Colab lib
    from google.colab import drive
    # Mount the drive
    drive.mount('/content/drive/', force_remount=True)
    sys.path.append(project_path)
    %cd $project_path

# Let's look at the sys path
print('Current working directory', os.getcwd())

Mounted at /content/drive/
/content/drive/My Drive/NER
Current working directory /content/drive/My Drive/NER


In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../NER"))

# Any results you write to the current directory are saved as output.
from tqdm import tqdm, trange


['ner.csv', 'ner_dataset.csv', 'xlmner.pt', 'restauranttrain.bio', 'restauranttest.bio', 'cache_dir', 'outputs', 'runs', 'modelner', 'modelxlmner.pt', 'drive', 'xlm_ner1.pt', 'name.pt']


In [7]:

import numpy as np # linear algebra
import pandas as pd

In [8]:
input_data = pd.read_csv("/content/drive/MyDrive/NER/ner_dataset.csv", encoding="latin1")
#input_data

In [9]:
input_data = input_data.fillna(method="ffill")
input_data.tail(10)
input_data

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O


In [10]:
words_list = list(set(input_data["Word"].values))
words_list[:10]

['sneaking',
 'hurricane-',
 'visit',
 'Beit',
 'pollsters',
 'Neskovic',
 'Re-elected',
 'Witnesses',
 'Baghaichhari',
 'dubious']

In [11]:
number_words = len(words_list); number_words # number of unique words in the corpus

35178

In [12]:
class RetrieveSentance(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        function = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(function)
        self.sentences = [s for s in self.grouped]
    
    def retrieve(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [13]:
Sentences = RetrieveSentance(input_data) 
Sentences

In [14]:
Sentences_list = [" ".join([s[0] for s in sent]) for sent in Sentences.sentences]
Sentences_list[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [15]:
len(Sentences_list) #number of sentences 

47959

In [16]:
labels = [[s[2] for s in sent] for sent in Sentences.sentences]
print(labels[0])

['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [17]:
labels [0] # list of lists of dimension (sentences,labels)

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-gpe',
 'O',
 'O',
 'O',
 'O',
 'O']

In [18]:
tags2vals = list(set(input_data["Tag"].values))
tag2idx = {t: i for i, t in enumerate(tags2vals)}

In [19]:
tags2vals # 17 kinds of tags 

['B-art',
 'I-nat',
 'I-geo',
 'I-gpe',
 'B-nat',
 'B-org',
 'B-eve',
 'I-per',
 'I-eve',
 'B-gpe',
 'B-per',
 'B-tim',
 'I-art',
 'I-tim',
 'I-org',
 'B-geo',
 'O']

In [20]:
tag2idx # indexing the tag 

{'B-art': 0,
 'B-eve': 6,
 'B-geo': 15,
 'B-gpe': 9,
 'B-nat': 4,
 'B-org': 5,
 'B-per': 10,
 'B-tim': 11,
 'I-art': 12,
 'I-eve': 8,
 'I-geo': 2,
 'I-gpe': 3,
 'I-nat': 1,
 'I-org': 14,
 'I-per': 7,
 'I-tim': 13,
 'O': 16}

In [21]:
pip install transformers

     |████████████████████████████████| 1.5MB 24.7MB/s 
     |████████████████████████████████| 2.9MB 54.7MB/s 
     |████████████████████████████████| 890kB 54.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a1559119bf07ece560c14e8bc6f71bf4082b6342f5ff91b81bf019d518e9d0b4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [22]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 15.9MB/s 


In [23]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import torch
from transformers import XLMRobertaTokenizer
import tensorflow
import sentencepiece


In [24]:
max_seq_len = 75 # tokens
batch_s = 32 # batch size

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [26]:
torch.cuda.get_device_name(0) 

'Tesla T4'

In [27]:
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base" )


In [28]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in Sentences_list]
print(tokenized_texts[0])

['▁Tho', 'usan', 'ds', '▁of', '▁demonstra', 'tors', '▁have', '▁marche', 'd', '▁through', '▁London', '▁to', '▁protest', '▁the', '▁war', '▁in', '▁Iraq', '▁and', '▁demand', '▁the', '▁withdraw', 'al', '▁of', '▁British', '▁tro', 'ops', '▁from', '▁that', '▁country', '▁', '.']


In [29]:
len(tokenized_texts)

47959

In [30]:
print(tokenized_texts[1])

['▁Iran', 'ian', '▁official', 's', '▁say', '▁they', '▁expect', '▁to', '▁get', '▁access', '▁to', '▁seal', 'ed', '▁sensitive', '▁parts', '▁of', '▁the', '▁plant', '▁Wednesday', '▁', ',', '▁after', '▁an', '▁I', 'A', 'EA', '▁sur', 've', 'illa', 'nce', '▁system', '▁begin', 's', '▁function', 'ing', '▁', '.']


In [31]:
X = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_seq_len, dtype="long", truncating="post", padding="post")

In [32]:
Y = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=max_seq_len, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [33]:
X.shape # (sentences, maximum sequence length)

(47959, 75)

In [34]:
Y.shape

(47959, 75)

In [35]:
X

array([[ 32238,  38504,   6468, ...,      0,      0,      0],
       [ 18721,   3378,  51521, ...,      0,      0,      0],
       [  1529,   9120,  78478, ...,      0,      0,      0],
       ...,
       [ 77168,    214,  18721, ...,      0,      0,      0],
       [ 66016,   7068,      6, ...,      0,      0,      0],
       [   581,  14098, 145704, ...,      0,      0,      0]])

In [36]:
Y

array([[16, 16, 16, ..., 16, 16, 16],
       [ 9, 16, 16, ..., 16, 16, 16],
       [16, 16, 11, ..., 16, 16, 16],
       ...,
       [16, 15, 16, ..., 16, 16, 16],
       [16, 16, 16, ..., 16, 16, 16],
       [16,  5, 14, ..., 16, 16, 16]])

In [37]:
attention_masks = [[float(i>0) for i in ii] for ii in X]

In [38]:
len(attention_masks) # list of lists of shape (sentences, labels )

47959

In [39]:
attention_masks[0]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [40]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, 
                                                            random_state=20, test_size=0.1)
Mask_train, Mask_valid, _, _ = train_test_split(attention_masks, X,
                                             random_state=20, test_size=0.1)

In [41]:
X_train = torch.tensor(X_train)
X_valid = torch.tensor(X_valid)
Y_train = torch.tensor(Y_train)
Y_valid = torch.tensor(Y_valid)
Mask_train = torch.tensor(Mask_train)
Mask_valid = torch.tensor(Mask_valid)

In [42]:
data_train = TensorDataset(X_train, Mask_train, Y_train)
data_train_sampler = RandomSampler(data_train)
DL_train = DataLoader(data_train, sampler=data_train_sampler, batch_size=batch_s)

data_valid = TensorDataset(X_valid, Mask_valid, Y_valid)
data_valid_sampler = SequentialSampler(data_valid)
DL_valid = DataLoader(data_valid, sampler=data_valid_sampler, batch_size=batch_s)

In [43]:
pip install fairseq

     |████████████████████████████████| 1.7MB 13.3MB/s 
     |████████████████████████████████| 71kB 11.6MB/s 
     |████████████████████████████████| 122kB 61.6MB/s 
     |████████████████████████████████| 112kB 61.3MB/s 
     |████████████████████████████████| 276kB 52.9MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp36-none-any.whl size=141231 sha256=f30f5c83b97b8e1249fc4732550d710131882e4aa1b21c295f0f6c904f08235f
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=7d6ee3dff3c5292097376b068b05216073c9fa8a1474eddc8212e2fc8478dc56
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built antlr4-python3-runtime PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

In [44]:
from transformers import XLMRobertaTokenizer,XLMRobertaForTokenClassification
import torch
import sentencepiece

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')


In [45]:
model=XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=17)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

In [46]:
print('Loading model to GPU...')

# Connect to the GPU.
device = torch.device('cuda')

# Report what GPU we were granted.
print('  GPU:', torch.cuda.get_device_name(0))

# Copy the initial model weights to the GPU.
desc = model.to(device)

print('    DONE.')

Loading model to GPU...
  GPU: Tesla T4
    DONE.


In [47]:
len(tag2idx)

17

In [48]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [49]:
pip install seqeval

     |████████████████████████████████| 51kB 6.1MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=381bdbd18dc70f2848800e8cfb4892e3ffcca5d23ae3c89442859513c43832d9
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [50]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [51]:
from transformers import AdamW

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 5e-6, # args.learning_rate
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [52]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs.
epochs = 5

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(DL_train) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [53]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))  

In [52]:
def good_update_interval(total_iters, num_desired_updates):
    '''
    This function will try to pick an intelligent progress update interval 
    based on the magnitude of the total iterations.

    Parameters:
      `total_iters` - The number of iterations in the for-loop.
      `num_desired_updates` - How many times we want to see an update over the 
                              course of the for-loop.
    '''
    # Divide the total iterations by the desired number of updates. Most likely
    # this will be some ugly number.
    exact_interval = total_iters / num_desired_updates

    # The `round` function has the ability to round down a number to, e.g., the
    # nearest thousandth: round(exact_interval, -3)
    #
    # To determine the magnitude to round to, find the magnitude of the total,
    # and then go one magnitude below that.

    # Get the order of magnitude of the total.
    order_of_mag = len(str(total_iters)) - 1

    # Our update interval should be rounded to an order of magnitude smaller. 
    round_mag = order_of_mag - 1

    # Round down and cast to an int.
    update_interval = int(round(exact_interval, -round_mag))

    # Don't allow the interval to be zero!
    if update_interval == 0:
        update_interval = 1

    return update_interval

In [53]:
import sys
if sys.version_info >= (3,):
    def input(__prompt: any = ...) -> str: ...
else:
    def input(__prompt: any = ...) -> any: ...
    def intern(__string: str) -> str: ...

In [54]:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42
epoch=2
import random
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0,epoch):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(DL_train):

        # Progress update every 100 batches.
        if step % 500 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(DL_train), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(DL_train)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
      
print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...
  Batch   500  of  1,349.    Elapsed: 0:04:12.
  Batch 1,000  of  1,349.    Elapsed: 0:08:26.

  Average training loss: 0.42
  Training epoch took: 0:11:24

======== Epoch 2 / 5 ========
Training...
  Batch   500  of  1,349.    Elapsed: 0:04:14.
  Batch 1,000  of  1,349.    Elapsed: 0:08:28.

  Average training loss: 0.25
  Training epoch took: 0:11:25

Training complete!


In [55]:
#               Validation
# ========================================
# After the completion of each training epoch, measure our performance on
# our validation set.

print("")
print("Running Validation...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
model.eval()

preds=[]
true=[]

# Tracking variables 
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# Evaluate data for one epoch
for batch in DL_valid:
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.

        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # Get the "logits" output by the model. The "logits" are the output
    # values prior to applying an activation function like the softmax.
    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    preds.append(logits)
    true.append(label_ids)
    # Calculate the accuracy for this batch of test sentences.
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    # Accumulate the total accuracy.
    eval_accuracy += tmp_eval_accuracy

    # Track the number of batches
    nb_eval_steps += 1

# Report the final accuracy for this validation run.
print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("  Validation took: {:}".format(format_time(time.time() - t0)))



Running Validation...
  Accuracy: 0.92
  Validation took: 0:00:23


In [54]:
PATH = "/content/drive/MyDrive/NER/name.pt"

In [57]:


# Save
torch.save(model,PATH) 

# # Load

# model.load_state_dict(torch.load(PATH))
# model.eval()

In [55]:
# Load
import torch
model1 = torch.load(PATH,map_location='cpu')
model1.eval()

XLMRobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Layer

In [56]:
test_sentence = """
Mr. Trump’s tweets began just moments after a Fox News report by Mike Tobin, a 
reporter for the network, about protests in Minnesota and elsewhere. 
"""



In [57]:
tokenized_sentence = tokenizer.encode(test_sentence)
input_ids = torch.tensor([tokenized_sentence])

In [58]:

with torch.no_grad():
    output = model1(input_ids)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

In [59]:
tag_values = list(set(input_data["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}
tag_values

['B-art',
 'I-nat',
 'I-geo',
 'I-gpe',
 'B-nat',
 'B-org',
 'B-eve',
 'I-per',
 'I-eve',
 'B-gpe',
 'B-per',
 'B-tim',
 'I-art',
 'I-tim',
 'I-org',
 'B-geo',
 'O',
 'PAD']

In [74]:
tag2=['B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim',
 'O',
 'PAD']

In [61]:
tag1=['B-nat',
 'B-geo',
 'B-art',
 'B-tim',
 'I-gpe',
 'I-tim',
 'I-org',
 'B-eve',
 'I-nat',
 'I-eve',
 'B-org',
 'I-per',
 'B-per',
 'O',
 'I-art',
 'I-geo',
 'B-gpe']

In [62]:
tag2idx = {t: i for i, t in enumerate(tag_values)}
tag2idx

{'B-art': 0,
 'B-eve': 6,
 'B-geo': 15,
 'B-gpe': 9,
 'B-nat': 4,
 'B-org': 5,
 'B-per': 10,
 'B-tim': 11,
 'I-art': 12,
 'I-eve': 8,
 'I-geo': 2,
 'I-gpe': 3,
 'I-nat': 1,
 'I-org': 14,
 'I-per': 7,
 'I-tim': 13,
 'O': 16,
 'PAD': 17}

In [63]:
# join bpe split tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(tag_values[label_idx])
        new_tokens.append(token)

In [64]:
for token, label in zip(new_tokens, new_labels):
    print("{}\t{}".format(label, token))

I-art	<s>
I-art	▁Mr
B-tim	.
I-tim	▁Trump
I-tim	’
I-tim	s
I-tim	▁tweet
I-tim	s
I-tim	▁began
I-tim	▁just
B-eve	▁moments
B-eve	▁after
I-tim	▁a
I-tim	▁Fox
I-art	▁News
B-tim	▁report
I-tim	▁by
I-tim	▁Mike
I-tim	▁To
I-tim	bin
I-tim	,
I-tim	▁a
I-tim	▁reporter
I-tim	▁for
I-nat	▁the
I-tim	▁network
I-tim	,
I-tim	▁about
I-tim	▁protest
I-tim	s
I-tim	▁in
I-tim	▁Minnesota
I-tim	▁and
I-tim	▁elsewhere
I-tim	.
I-tim	</s>


In [65]:
import nltk
from nltk import word_tokenize

In [83]:
    def generate_ner(sentence):
        tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
        model = torch.load(PATH, map_location='cpu')
        tokenized_sentence = tokenizer.encode(test_sentence)
        input_ids = torch.tensor([tokenized_sentence])
        with torch.no_grad():
            output = model(input_ids)
        label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
        tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
        new_tokens, new_labels = [], []
        for token, label_idx in zip(tokens, label_indices[0]):
            if token.startswith("##"):
                new_tokens[-1] = new_tokens[-1] + token[2:]
            else:
                new_labels.append(tag1[label_idx])
                new_tokens.append(token)
        prediction= [{"word":token,"tag":label}for token, label in zip(new_tokens, new_labels)]
        return prediction

In [84]:
generate_ner(test_sentence)

[{'tag': 'B-per', 'word': '<s>'},
 {'tag': 'B-per', 'word': '▁Mr'},
 {'tag': 'I-per', 'word': '.'},
 {'tag': 'O', 'word': '▁Trump'},
 {'tag': 'O', 'word': '’'},
 {'tag': 'O', 'word': 's'},
 {'tag': 'O', 'word': '▁tweet'},
 {'tag': 'O', 'word': 's'},
 {'tag': 'O', 'word': '▁began'},
 {'tag': 'O', 'word': '▁just'},
 {'tag': 'I-org', 'word': '▁moments'},
 {'tag': 'I-org', 'word': '▁after'},
 {'tag': 'O', 'word': '▁a'},
 {'tag': 'O', 'word': '▁Fox'},
 {'tag': 'B-per', 'word': '▁News'},
 {'tag': 'I-per', 'word': '▁report'},
 {'tag': 'O', 'word': '▁by'},
 {'tag': 'O', 'word': '▁Mike'},
 {'tag': 'O', 'word': '▁To'},
 {'tag': 'O', 'word': 'bin'},
 {'tag': 'O', 'word': ','},
 {'tag': 'O', 'word': '▁a'},
 {'tag': 'O', 'word': '▁reporter'},
 {'tag': 'O', 'word': '▁for'},
 {'tag': 'B-geo', 'word': '▁the'},
 {'tag': 'O', 'word': '▁network'},
 {'tag': 'O', 'word': ','},
 {'tag': 'O', 'word': '▁about'},
 {'tag': 'O', 'word': '▁protest'},
 {'tag': 'O', 'word': 's'},
 {'tag': 'O', 'word': '▁in'},
 {'ta